#### Tuning XGB Adjuster: Adult Data

In [ ]:
from aif360.algorithms.inprocessing.xgb_adversarial_debiasing import XGBFairnessAdjuster
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import (
    load_preproc_data_adult,
)

#### Load dataset and set options

In [ ]:
# Get the dataset and split into train and test
dataset_orig = load_preproc_data_adult(protected_attributes=["sex", "race"])
dataset_orig_train, dataset_orig_test = dataset_orig.split(
    [0.9], shuffle=True, seed=1234
)
privileged_groups = [{"sex": 1}]
unprivileged_groups = [{"sex": 0}]

### Learn plan classifier without debiasing

In [ ]:
base_settings = {
    "time_budget": 60,  # total running time in seconds
    "metric": "log_loss",
}

In [ ]:
plain_model = XGBFairnessAdjuster(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    seed=1234,
    debias=False,
    tune_hyperparameters_base=True,
    tuning_settings_base=base_settings,
    tune_hyperparameters_adjuster=False,
)

In [ ]:
plain_model.fit(dataset=dataset_orig_train, test_dataset=dataset_orig_test)

### Apply in-processing algorithm based on adversarial learning

In [ ]:
debiased_model = XGBFairnessAdjuster(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    seed=1234,
    debias=True,
    adversary_loss_weight=1.0,
    tune_hyperparameters_base=True,
    tuning_settings_base=base_settings,
    tune_hyperparameters_adjuster=False,
    debug=False,
    task="classification",
)

In [ ]:
debiased_model.fit(dataset=dataset_orig_train, test_dataset=dataset_orig_test)

In [ ]:
import matplotlib.pyplot as plt

bins = 20
probs_ad = debiased_model.predict_proba(dataset_orig_test)
base_probs = plain_model.predict_proba(dataset_orig_test)
# plt.hist(adjusted_probs, bins=bins, histtype="step", label="Fairness Adjuster")
plt.hist(probs_ad, bins=bins, histtype="step", label="Debiased")
plt.hist(base_probs, bins=bins, histtype="step", label="Not Debiased")
plt.legend()
plt.show()

In [ ]:
plain_model.base_estimator.best_config

In [ ]:
debiased_model.base_estimator.best_config